In [2]:
!pip install nest_asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Import Module

import requests
import nest_asyncio
import asyncio
from bs4 import BeautifulSoup
import time
import re
from tqdm import tqdm

# Synchronous Programming

In [7]:
results = []
s = time.time()

for page in tqdm(range(1,10)):
    url = f'https://movie.naver.com/movie/point/af/list.naver?&page={page}'
    #get : request로 url의  html문서의 내용 요청
    html = requests.get(url)
    #html을 받아온 문서를 .content로 지정 후 soup객체로 변환
    soup = BeautifulSoup(html.content,'html.parser')
    #find_all : 지정한 태그의 내용을 모두 찾아 리스트로 반환
    reviews = soup.find_all("td",{"class":"title"})
    results.append(reviews)

e = time.time()
print("{0:.2f}초 걸렸습니다".format(e - s))

100%|██████████| 9/9 [00:07<00:00,  1.14it/s]

7.90초 걸렸습니다


# Asynchronous Programming

In [9]:
s = time.time()
results = []
urls = [f'https://movie.naver.com/movie/point/af/list.naver?&page={page}' for page in range(1, 10)]

nest_asyncio.apply()

#---------------------- A 파트 ------------------------
async def getpage(url):
    req = await loop.run_in_executor(None, requests.get, url)
    html = req.text
    soup = await loop.run_in_executor(None, BeautifulSoup, html, 'html.parser')
    reviews = soup.find_all("td",{"class":"title"})
    results.append(reviews)
    return

#---------------------- B 파트 ------------------------ 
async def main():
    fts = [asyncio.ensure_future(getpage(u)) for u in urls]
    r = await asyncio.gather(*fts)
    global results
    results = r

#---------------------- C 파트 ------------------------
loop = asyncio.get_event_loop()
loop.run_until_complete(main())
loop.close
e = time.time()

print("{0:.2f}초 걸렸습니다".format(e - s))

1.82초 걸렸습니다


- Asynchronous Programming
  - 일반적인 코드의 경우 IO 사이의 대기시간이 존재한다. 
  - 비동기식 프로그래밍의 경우 이 대기시간을 멀티태스킹으로 처리하여 불필요한 대기시간을 줄일 수 있다.
  - 이를 가능하게 해주는 함수를 코루틴(Coroutine)이라 한다.
  - 위 경우처럼 여러 웹페이지를 불러올 때 비동기식으로 코드를 작성하는 경우 여러 페이지를 로딩될 때까지 기다리지 않고 바로 다음 웹페이지를 불러오며 매우 빠른 속도로 모든 웹페이지를 불러올 수 있다.

# Detail of C Part

- C Part는 위 코드에서 가장 먼저 실행되는 파트이다.
- 비동기식 코드는 대기시간이 없도록 main() 함수(코루틴)가 종료될 때까지 루프를 만들어줘야한다.


In [10]:
#---------------------- C 파트 ------------------------
loop = asyncio.get_event_loop() # 루프 생성
loop.run_until_complete(main()) # main() 코루틴 종료시점을 루프 종료시점으로 지정
loop.close # 루프 종료

<bound method _UnixSelectorEventLoop.close of <_UnixSelectorEventLoop running=True closed=False debug=False>>

# Detail Of B Part

- Python을 통해 코루틴을 작성하기 위해서 def 구문 앞에 async 구문을 붙여줘야한다.
- fts 변수는 데이터 크롤링을 위한 url 주소값으로 future 객체로 만들어줘야한다.
- future 객체는 코루틴에 의해 실행되는 계획서이다.
- await asyncio.gather(*fts) 구문을 통해 계획서를 실질적으로 실행한다.
 - *는 리스트를 해체해서 fts 변수의 값을 전달하는 역할을 한다.

In [ ]:
#---------------------- B 파트 ------------------------ 
async def main():
    fts = [asyncio.ensure_future(getpage(u)) for u in urls]
    r = await asyncio.gather(*fts)
    global results
    results = r

# Detail Of A Part

- A Part는 Main 코루틴이다.
- await은 코루틴을 실행하는 예약어로 이를 이용해야 다른 코루틴을 실행할 수 있다.
- loop.run_in_executor()는 실행한 코루틴이 된다.
- Python의 모든 함수가 비동기식으로 만들어지지 않아 지원 가능한 함수만 코루틴으로써 실행할수 있다. 이 때 loop.run_in_executor() 코루틴을 사용하면 일반 함수를 코루틴과 동일하게 사용 가능하다.

In [ ]:
async def getpage(url):
    req = await loop.run_in_executor(None, requests.get, url)
    html = req.text
    soup = await loop.run_in_executor(None, BeautifulSoup, html, 'html.parser')
    reviews = soup.find_all("td",{"class":"title"})
    results.append(reviews)
    return